In [1]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from shapely.geometry import Point
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from patsy import dmatrices

#### Reads in csv with data from question 2 and converts cells into appropriate objects 

In [4]:
burglary_by_tract = pd.read_csv("../../geospatial_api-silver-sharks/data/burglary_by_tract.csv")

burglary_by_tract['tract'] = burglary_by_tract['tract'].astype(str)
burglary_by_tract.head()
burglary_by_tract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           773 non-null    int64  
 1   Unnamed: 0.1         773 non-null    int64  
 2   incident_reported    773 non-null    object 
 3   offense_description  773 non-null    object 
 4   latitude             773 non-null    float64
 5   longitude            773 non-null    float64
 6   geometry             773 non-null    object 
 7   COUNTYFP             773 non-null    int64  
 8   tract                773 non-null    object 
 9   NAME                 773 non-null    float64
dtypes: float64(3), int64(3), object(4)
memory usage: 60.5+ KB


### Imports Census data

In [9]:
with open('../data/data_census.json') as fi:
    credentials = json.load(fi)
       
api_key = credentials['api_key']


In [10]:
endpoint_pop = "https://api.census.gov/data/2019/acs/acs5"
params_pop = {
       'get' : 'NAME,B01001_001E',
       'for' : 'tract:*',
       'in'  : ['state:47', 'county:037'],
       'key' : api_key
}

endpoint_income = "https://api.census.gov/data/2019/acs/acs5/subject"
params_income = {
       'get' : 'NAME,S1901_C01_012E',
       'for' : 'tract:*',
       'in'  : ['state:47', 'county:037'],
       'key' : api_key
}


In [11]:
response_pop = requests.get(endpoint_pop, params=params_pop)

response_income = requests.get(endpoint_income, params=params_income)

In [12]:
res_pop = response_pop.json()
population = pd.DataFrame(res_pop)
#changes column names
population.columns = population.iloc[0]
population = population[1:]
display(population.head())

res_income = response_income.json()
med_income = pd.DataFrame(res_income) 
#changes colum names 
med_income.columns = med_income.iloc[0]
med_income = med_income[1:]
display(med_income.head())

,NAME,B01001_001E,state,county,tract
1,"Census Tract 156.31, Davidson County, Tennessee",12176,47,037,015631
2,"Census Tract 158.04, Davidson County, Tennessee",4098,47,037,015804
3,"Census Tract 177.01, Davidson County, Tennessee",2466,47,037,017701
4,"Census Tract 177.02, Davidson County, Tennessee",5210,47,037,017702
5,"Census Tract 183.01, Davidson County, Tennessee",8254,47,037,018301


,NAME,S1901_C01_012E,state,county,tract
1,"Census Tract 156.31, Davidson County, Tennessee",75579,47,037,015631
2,"Census Tract 158.04, Davidson County, Tennessee",45129,47,037,015804
3,"Census Tract 177.01, Davidson County, Tennessee",134786,47,037,017701
4,"Census Tract 177.02, Davidson County, Tennessee",107813,47,037,017702
5,"Census Tract 183.01, Davidson County, Tennessee",87591,47,037,018301


### Merges census data together and renames columns

In [13]:
pop_and_income = pd.merge(population, med_income, how='inner')
pop_and_income = pop_and_income.rename(columns={'B01001_001E': 'population',
                                               'S1901_C01_012E':'median_income'})
pop_and_income['tract'] = pop_and_income['tract'].astype(np.int64)
pop_and_income['tract'] = pop_and_income['tract'].astype(str)
pop_and_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161 entries, 0 to 160
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   NAME           161 non-null    object
 1   population     161 non-null    object
 2   state          161 non-null    object
 3   county         161 non-null    object
 4   tract          161 non-null    object
 5   median_income  161 non-null    object
dtypes: object(6)
memory usage: 8.8+ KB


### Merges Population and Income data with burglaries by tract. 
#### Also, deleates duplicate columns 

In [14]:
burglaries_and_var = pd.merge(burglary_by_tract, pop_and_income, on='tract')
burglaries_and_var = burglaries_and_var.drop(columns=['latitude', 'longitude','NAME_x','NAME_y'])
burglaries_and_var.head()

,Unnamed: 0,Unnamed: 0.1,incident_reported,offense_description,geometry,COUNTYFP,tract,population,state,county,median_income
0,0,199,2021-01-13,BURGLARY- AGGRAVATED,POINT (-86.95 36.06),37,18407,5001,47,037,105917
1,217,14375,2021-03-05,BURGLARY- AGGRAVATED,POINT (-86.98999999999999 36.07),37,18407,5001,47,037,105917
2,1,310,2021-01-14,BURGLARY- AGGRAVATED,POINT (-86.75 36.19),37,11700,5971,47,037,68889
3,42,2779,2021-01-22,BURGLARY- AGGRAVATED,POINT (-86.73999999999999 36.19),37,11700,5971,47,037,68889
4,50,3623,2021-01-25,BURGLARY- AGGRAVATED,POINT (-86.75 36.19),37,11700,5971,47,037,68889


### Creates DataFrame that is a count of the # of burglaries per tract 
#### Merges back in population and income to those tracts

In [15]:
#tract_counts = pd.DataFrame(burglaries_and_var['tract'].value_counts())
tract_counts = pd.DataFrame(burglaries_and_var['tract'].value_counts())
tract_counts = tract_counts.rename(columns={'tract':'burglary_count'})

pop_income = pd.DataFrame(burglaries_and_var[['tract', 'population', 'median_income']])
pop_income = pop_income.set_index('tract')

tract_pop_income = pd.merge(pop_income, tract_counts, left_index=True, right_index=True)
tract_pop_income = tract_pop_income.drop_duplicates(keep='first')

tract_pop_income.reset_index(level=0, inplace=True)
tract_pop_income = tract_pop_income.rename(columns={'index': 'tract'})

#Filters out tracts with a population or median income with a below 0 value 
tract_pop_income[['population', 'median_income']] = tract_pop_income[['population', 'median_income']].apply(pd.to_numeric)
tract_pop_income = tract_pop_income[tract_pop_income['population'] >= 0]
tract_pop_income = tract_pop_income[tract_pop_income['median_income'] >= 0]

tract_pop_income.head()

,tract,population,median_income,burglary_count
0,10104,2955,64844,5
1,10105,4435,64688,3
2,10106,3397,63864,5
3,10201,4157,62829,3
4,10301,3395,51918,1


#### Adds burglaries per 1000 column for possible use in choropleth 

In [16]:
tract_pop_income['burg_per_1000'] = ((tract_pop_income['burglary_count'] / tract_pop_income['population']) * 1000)
tract_pop_income['log_population'] = np.log(tract_pop_income['population'])
tract_pop_income.head()

,tract,population,median_income,burglary_count,burg_per_1000,log_population
0,10104,2955,64844,5,1.692047,7.991254
1,10105,4435,64688,3,0.676437,8.397283
2,10106,3397,63864,5,1.471887,8.130648
3,10201,4157,62829,3,0.721674,8.332549
4,10301,3395,51918,1,0.294551,8.130059


#### Converts to csv file for next steps

In [18]:
tract_pop_income.to_csv("../../geospatial_api-silver-sharks/data/tract_pop_income.csv")